In [22]:
from datetime import datetime, timedelta
from ordered_set import OrderedSet
from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, ConstantVelocity
from stonesoup.types.groundtruth import GroundTruthPath, GroundTruthState
from utils import inward_velocity_vector, random_edge_position
import random 
import numpy as np
seed = 1992
np.random.seed(seed)

q = 0.1
num_steps = 200
FOV = [0,500]

birth_rate = 0.03  
death_probability = 0
time_stepsize = 1
start_time = datetime.now().replace(microsecond=0)

timestep_size = timedelta(seconds=1)
truths = OrderedSet()  # Truths across all time
current_truths = set() # Truths alive at the current time

# Define a transition model with some noise
transition_model = CombinedLinearGaussianTransitionModel([ConstantVelocity(q)])

# Calculate the FOV center
fov_center = (FOV[1]-FOV[0])/2

timesteps = []
for k in range(num_steps):
    timesteps.append(start_time + timedelta(seconds=k))

    # Death (Only for truths within FOV)
    for truth in current_truths.copy():
        if truth[-1].state_vector[0] < FOV[0] or truth[-1].state_vector[0] > FOV[1]: 
            current_truths.remove(truth)

    # Update truths
    for truth in current_truths:
        truth.append(GroundTruthState(transition_model.function(truth[-1], noise=True, time_interval=timedelta(seconds=1)), timestamp=timesteps[k]))

    # Birth based on Poisson process
    num_births = np.random.poisson(birth_rate)
    
    for _ in range(num_births):
        initial_position = random_edge_position(FOV)
        initial_velocity = inward_velocity_vector(initial_position, fov_center)  # Inward velocity
        state = GroundTruthState([initial_position, initial_velocity], timestamp=timesteps[k])
        truth = GroundTruthPath([state])
        current_truths.add(truth)
        truths.add(truth)

In [23]:
from scipy.stats import uniform
from stonesoup.types.detection import TrueDetection
from stonesoup.types.detection import Clutter
from stonesoup.models.measurement.linear import LinearGaussian

local_clutter_rate = 0 # optionally 0 if increased clutter around target is NOT desired.
local_clutter_range = 10
global_clutter_rate = 2
clutter_area = FOV
r = 0.1
prob_detect = 0.90
clutter_spatial_density = global_clutter_rate / (FOV[1]-FOV[0])

measurement_model = LinearGaussian(
    ndim_state=2,
    mapping=(0,),
    noise_covar=np.array([[r]])
    )
all_measurements = []

for k in range(num_steps):
    measurement_set = set()
    timestamp = start_time + timedelta(seconds=k)

    # Generate general background clutter in all of FOV
    num_clutter = np.random.poisson(global_clutter_rate)
    for _ in range(num_clutter):
        x = uniform.rvs(FOV[0], FOV[1])
        measurement_set.add(Clutter(np.array([[x]]), timestamp = timestamp, measurement_model = measurement_model))
    for truth in truths:
        try:
            truth_state = truth[timestamp]
        except IndexError:
            # This truth not alive at this time.
            continue
        # Generate actual detection from the state with a 10% chance that no detection is received.
        if np.random.rand() <= prob_detect:
            # Generate actual detection from the state
            measurement = measurement_model.function(truth_state, noise=True)
            measurement_set.add(TrueDetection(state_vector=measurement,
                                              groundtruth_path=truth,
                                              timestamp=truth_state.timestamp,
                                              measurement_model=measurement_model))

        # Generate clutter at this time-step around the truth
        truth_x = truth_state.state_vector[0]
        num_clutter = np.random.poisson(local_clutter_rate)
        for _ in range(num_clutter):
            x = uniform.rvs(truth_x - local_clutter_range, 2*local_clutter_range)
            measurement_set.add(Clutter(np.array([[x]]), timestamp=timestamp,
                                        measurement_model=measurement_model))


        

    all_measurements.append(measurement_set)

In [24]:
from CustomDeleter import CustomDeleter
deleter = CustomDeleter(covar_trace_thresh=50, fov = FOV)

from stonesoup.predictor.kalman import KalmanPredictor
transition_model = CombinedLinearGaussianTransitionModel([ConstantVelocity(1)])
predictor = KalmanPredictor(transition_model)

from stonesoup.updater.kalman import KalmanUpdater
measurement_model = LinearGaussian(ndim_state=2,mapping=(0,),noise_covar=np.array([[1000000]]))
updater = KalmanUpdater(measurement_model)

from stonesoup.hypothesiser.distance import DistanceHypothesiser
from stonesoup.measures import Mahalanobis
hypothesiser_initiator = DistanceHypothesiser(predictor, updater, measure=Mahalanobis(), missed_distance=3)

from stonesoup.dataassociator.neighbour import GNNWith2DAssignment
data_associator_initiator = GNNWith2DAssignment(hypothesiser_initiator)



from stonesoup.types.state import GaussianState
from CustomInitiator import RestrictedInitiator

initiator = RestrictedInitiator(
    prior_state = None,
    measurement_model=measurement_model,
    deleter=deleter,
    data_associator=data_associator_initiator,
    updater=updater,
    min_points=5,
    FOV = FOV,
    range = 20
    )

from stonesoup.hypothesiser.probability import PDAHypothesiser
from stonesoup.dataassociator.probability import JPDA
hypothesiser = PDAHypothesiser(
    predictor=predictor,
    updater=updater,
    clutter_spatial_density=clutter_spatial_density,
    prob_detect=prob_detect
 
)

data_associator = JPDA(hypothesiser=hypothesiser)

from tracker import JPDAtracker
tracker = JPDAtracker(
    updater = updater,
    data_associator = data_associator,
    initiator = initiator,
    deleter = deleter,
    random_seed = seed
)



In [25]:
all_tracks1 = tracker.track(all_measurements, start_time)

In [26]:
from utils import Plotter
plotter = Plotter(truths=truths, all_measurements=all_measurements, all_tracks=all_tracks1, FOV = FOV)

In [27]:
plotter.plot_track_pos(plotly=True)

In [28]:
plotter.plot_track_vel(plotly=True)